# 1st Level - GLM Model Fitting 

In [1]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
import numpy as np
from PIL import Image

import os
import re

In [2]:
def gather_subject_info(tsv_filename):
    """
    a function for gathering individual subject info
    this is the event file for each subject to which we will model
    our GLM.
    """
    import pandas as pd
    from nipype.interfaces.base import Bunch
    
    trialinfo = pd.read_table(tsv_filename)
    conditions = []
    onsets = []
    durations = []
    weights = []

    for group in trialinfo.groupby('stimulus'):
        conditions.append(group[0])
        onsets.append(list(group[1].onset)) 
        durations.append(list(group[1].duration))
        weights.append(list(group[1].weight))

    subject_info = Bunch(conditions=conditions,
                        onsets=onsets,
                        durations=durations,
                        amplitudes=weights)

    return subject_info

In [3]:
# uncomment below to verify conditions
#gather_subject_info(r'/home/anakin/Desktop/ds000113/sub-01/ses-movie/func/sub-01_task-sync_run-2_events.tsv')

We will define first level contrasts, these will include all condition contrasts (condition > baseline) and controlled contrasts (condition >  other condition).  
We assume 'General synchrony' is the baseline of percieved synchrony in a specific domain, therefore we control our social conditions  
(affect, gaze and touch) using a simple subtraction of each condition from basline (general).  
FSL computes a one sided t-test for each contrast, so we will compute both directions.


In [4]:
data = r"/home/anakin/Desktop/ds000113/output/preprocessing" # where we saved our preprocessing output
output = r"/home/anakin/Desktop/ds000113/output/1st_level" # where we would like to save 1st level output
brain_mask = r"/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz" 
# number of subs (8), number of runs (2) and TR initialization
num_of_subjects = 10 
num_of_runs = 2
excluded = [7,8]
subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1) if i not in excluded]
TR = 2
# conditions (aka predictors/regressors/events..)
cont01 = ['normdiff', 'T', ['normdiff'], [1]]
cont02 = ['shot', 'T', ['shot'], [1]]
cont03 = ['face', 'T', ['face'], [1]]
cont04 = ['rms', 'T', ['rms'], [1]]
cont05 = ['speech', 'T', ['speech'], [1]]
cont06 = ['gaze',   'T', ['gaze'], [1]]
cont07 = ['touch', 'T', ['touch'], [1]]
cont08 = ['affect', 'T', ['affect'], [1]]
cont09 = ['general', 'T', ['general'], [1]]
# contrasts 
cont10 = ['normdiff > rms', 'T', ['normdiff', 'rms'], [1, -1]]
cont11 = ['rms > normdiff', 'T', ['normdiff', 'rms'], [-1,1]]
cont12 = ['face > speech', 'T', ['face', 'speech'], [1, -1]]
cont13 = ['speech > face', 'T', ['face', 'speech'], [-1, 1]]
cont14 = ['general > other', 'T', ['general', 'gaze', 'touch', 'affect'], [3, -1,-1,-1]]
cont15 = ['other > general', 'T', ['general', 'gaze', 'touch', 'affect'], [-3, 1,1,1]]

contrasts = [cont01, cont02, cont03, cont04, cont05,cont06, cont07, cont08, cont09, cont10,
             cont11, cont12, cont13, cont14, cont15]

In [5]:
# infosource node
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]
# checking subject list
infosource.iterables

[('subject_list', ['01', '02', '03', '04', '05', '06', '09', '10'])]

In [6]:
# selectfiles and datasink nodes
templates = {"func"       : "{subject_list}/smooth/{subject_list}/smooth_run_*/sub-{subject_list}_task-sync_run-*_bold_maths_roi_mcf_warp_smooth.nii.gz",
             "real_params": "{subject_list}/realign/{subject_list}/realign_*/sub-{subject_list}_task-sync_run-*_bold_maths_roi_mcf.nii.gz.par",
             "conditions" : "{subject_list}/conditions/sub-{subject_list}_task-sync_run-*_events.tsv",
             "outliers"   : "{subject_list}/art/{subject_list}/art_run_*/art.sub-{subject_list}_task-sync_run-*_bold_maths_roi_mcf_outliers.txt"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data

substitutions = []
substitutions += [("_subject_list_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1)]
substitutions += [("_modelestimate{:d}".format(i), "modelestimate_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]
substitutions += [("_modelgen{:d}".format(i), "modelgen_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]
substitutions += [("_level1design{:d}".format(i), "level1design_run_{:d}".format(i+1)) for i in range(0,num_of_runs)]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [7]:
# connecting nodes 
getsubjectinfo = MapNode(Function(input_names=["tsv_filename"],output_names=['subject_info'],
                                  function=gather_subject_info),
                                  iterfield = ["tsv_filename"],
                                  name='getsubjectinfo')

mask_functional = MapNode(fsl.maths.ApplyMask(),name="mask_functional",iterfield=["in_file"])
mask_functional.inputs.mask_file = brain_mask

specify_model = MapNode(SpecifyModel(),name="specify_model",
                        iterfield=["functional_runs","subject_info","realignment_parameters","outlier_files"])
specify_model.inputs.input_units = 'secs'
specify_model.inputs.time_repetition = TR
specify_model.inputs.high_pass_filter_cutoff = 100.              
specify_model.inputs.parameter_source = "FSL"

level1design = MapNode(fsl.Level1Design(), name="level1design",iterfield = ["session_info"])# make the model
level1design.inputs.bases = {'dgamma':{'derivs': True}} #change if not intrested in Temporal derivatives in model
level1design.inputs.interscan_interval = TR
level1design.inputs.model_serial_correlations=True
level1design.inputs.contrasts=contrasts

modelgen = MapNode(fsl.FEATModel(),name='modelgen',iterfield = ["fsf_file","ev_files"]) # transform to fsl model

model_threshold = MapNode(fsl.ImageStats(),name="model_threshold",iterfield=["in_file"])
model_threshold.inputs.op_string = "-P 10"

modelestimate = MapNode(fsl.FILMGLS(), name='modelestimate',iterfield=["in_file","design_file","tcon_file","threshold"]) #run the model
modelestimate.inputs.smooth_autocorr=True 
modelestimate.inputs.mask_size=5 

In [8]:
# creating the workflow and connecting nodes
first_level_analysis = Workflow(name="first_level_analysis")

first_level_analysis.connect(infosource, "subject_list", selectfiles, "subject_list")
first_level_analysis.connect(selectfiles, "conditions", getsubjectinfo, "tsv_filename")
first_level_analysis.connect(selectfiles, "func", mask_functional, "in_file")
first_level_analysis.connect(selectfiles, "real_params", specify_model, "realignment_parameters")
first_level_analysis.connect(selectfiles, "outliers", specify_model, "outlier_files")
first_level_analysis.connect(getsubjectinfo,"subject_info",specify_model,"subject_info")
first_level_analysis.connect(mask_functional, "out_file", specify_model, "functional_runs")
first_level_analysis.connect(mask_functional, "out_file", model_threshold, "in_file")
first_level_analysis.connect(specify_model,"session_info",level1design,"session_info")
first_level_analysis.connect(level1design,"fsf_files",modelgen,"fsf_file")
first_level_analysis.connect(level1design,"ev_files",modelgen,"ev_files")
first_level_analysis.connect(mask_functional, "out_file", modelestimate, "in_file")
first_level_analysis.connect(model_threshold, "out_stat", modelestimate, "threshold")
first_level_analysis.connect(modelgen,"design_file",modelestimate,"design_file")
first_level_analysis.connect(modelgen,"con_file",modelestimate,"tcon_file")

In [9]:
first_level_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                              dotfilename= os.path.join(output,"first_level_analysis_workflow.dot"))

# visualize workflow
img = Image.open(os.path.join(output,"first_level_analysis_workflow.png"))
img.show()

231012-14:59:26,128 nipype.workflow INFO:
	 Generated workflow graph: /home/anakin/Desktop/ds000113/output/1st_level/first_level_analysis_workflow.png (graph2use=flat, simple_form=True).


In [10]:
# connect to datasink node
first_level_analysis.connect(level1design,"fsf_files",datasink,"level1design.@fsf_files")
first_level_analysis.connect(level1design,"ev_files",datasink,"level1design.@ev_files")

first_level_analysis.connect(modelgen,"design_file",datasink,"model.@design_file")
first_level_analysis.connect(modelgen,"design_cov",datasink,"model.@design_cov")
first_level_analysis.connect(modelgen,"design_image",datasink,"model.@design_matrix")

first_level_analysis.connect(modelestimate,"copes",datasink,"modelestimate.@copes")
first_level_analysis.connect(modelestimate,"varcopes",datasink,"modelestimate.@varcopes")
first_level_analysis.connect(modelestimate,"tstats",datasink,"modelestimate.@tstats")
first_level_analysis.connect(modelestimate,"zstats",datasink,"modelestimate.@zstats")
first_level_analysis.connect(modelestimate,"dof_file",datasink,"modelestimate.@dof_file")

In [11]:
first_level_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 1}) # on my laptop - took 1 hour per sub

231012-14:59:26,267 nipype.workflow INFO:
	 Workflow first_level_analysis settings: ['check', 'execution', 'logging', 'monitoring']
231012-14:59:26,373 nipype.workflow INFO:
	 Running in parallel.
231012-14:59:26,578 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-14:59:26,649 nipype.workflow INFO:
	 [Node] Setting-up "first_level_analysis.selectfiles" in "/tmp/tmpag2g12c9/first_level_analysis/_subject_list_01/selectfiles".
231012-14:59:26,659 nipype.workflow INFO:
	 [Node] Executing "selectfiles" <nipype.interfaces.io.SelectFiles>
231012-14:59:26,677 nipype.workflow INFO:
	 [Node] Finished "selectfiles", elapsed time 0.015224s.
231012-14:59:28,580 nipype.workflow INFO:
	 [Job 0] Completed (first_level_analysis.selectfiles).
231012-14:59:28,590 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 9 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-14:59:28,656 nipype.workflow IN

231012-15:00:12,871 nipype.workflow INFO:
	 [Node] Finished "_mask_functional0", elapsed time 22.194228s.
231012-15:00:14,628 nipype.workflow INFO:
	 [Job 74] Completed (_mask_functional0).
231012-15:00:14,634 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 29 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-15:00:14,689 nipype.workflow INFO:
	 [Node] Setting-up "_mask_functional1" in "/tmp/tmpxzxu78if/first_level_analysis/_subject_list_01/mask_functional/mapflow/_mask_functional1".
231012-15:00:14,693 nipype.workflow INFO:
	 [Node] Executing "_mask_functional1" <nipype.interfaces.fsl.maths.ApplyMask>
231012-15:00:16,629 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 28 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * _mask_functional1
231012-15:00:42,281 nipype.workflow INFO:
	 [Node] Finished "_mask_functional1", elapsed time 27.585406s.
231012-15:00:42,656 nipyp

231012-15:01:51,996 nipype.workflow INFO:
	 [Node] Finished "_getsubjectinfo0", elapsed time 0.151406s.
231012-15:01:53,842 nipype.workflow INFO:
	 [Job 80] Completed (_getsubjectinfo0).
231012-15:01:53,848 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 31 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-15:01:53,930 nipype.workflow INFO:
	 [Node] Setting-up "_getsubjectinfo1" in "/tmp/tmpmw7scjew/first_level_analysis/_subject_list_03/getsubjectinfo/mapflow/_getsubjectinfo1".
231012-15:01:53,933 nipype.workflow INFO:
	 [Node] Executing "_getsubjectinfo1" <nipype.interfaces.utility.wrappers.Function>
231012-15:01:54,111 nipype.workflow INFO:
	 [Node] Finished "_getsubjectinfo1", elapsed time 0.175768s.
231012-15:01:55,845 nipype.workflow INFO:
	 [Job 81] Completed (_getsubjectinfo1).
231012-15:01:55,849 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 31 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-15:01:55,956 nipype.w

231012-15:03:31,794 nipype.workflow INFO:
	 [Node] Finished "_mask_functional0", elapsed time 25.169727s.
231012-15:03:32,528 nipype.workflow INFO:
	 [Job 86] Completed (_mask_functional0).
231012-15:03:32,530 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 29 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-15:03:32,639 nipype.workflow INFO:
	 [Node] Setting-up "_mask_functional1" in "/tmp/tmpx5x_e0cv/first_level_analysis/_subject_list_04/mask_functional/mapflow/_mask_functional1".
231012-15:03:32,649 nipype.workflow INFO:
	 [Node] Executing "_mask_functional1" <nipype.interfaces.fsl.maths.ApplyMask>
231012-15:03:34,528 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 28 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * _mask_functional1
231012-15:04:00,643 nipype.workflow INFO:
	 [Node] Finished "_mask_functional1", elapsed time 27.991821s.
231012-15:04:02,557 nipyp

231012-15:05:16,31 nipype.workflow INFO:
	 [Node] Finished "_getsubjectinfo0", elapsed time 0.151427s.
231012-15:05:17,878 nipype.workflow INFO:
	 [Job 92] Completed (_getsubjectinfo0).
231012-15:05:17,884 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 31 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-15:05:18,17 nipype.workflow INFO:
	 [Node] Setting-up "_getsubjectinfo1" in "/tmp/tmpup02dlkv/first_level_analysis/_subject_list_06/getsubjectinfo/mapflow/_getsubjectinfo1".
231012-15:05:18,20 nipype.workflow INFO:
	 [Node] Executing "_getsubjectinfo1" <nipype.interfaces.utility.wrappers.Function>
231012-15:05:18,197 nipype.workflow INFO:
	 [Node] Finished "_getsubjectinfo1", elapsed time 0.174642s.
231012-15:05:19,880 nipype.workflow INFO:
	 [Job 93] Completed (_getsubjectinfo1).
231012-15:05:19,885 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 31 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231012-15:05:20,28 nipype.workf

...

231012-23:16:25,251 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/level1design/_subject_list_10/_level1design1/ev_Realign3_0_37.txt -> /home/anakin/Desktop/ds000113/output/1st_level/level1design/10/level1design_run_2/ev_Realign3_0_37.txt
231012-23:16:25,253 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/level1design/_subject_list_10/_level1design1/ev_Realign4_0_38.txt -> /home/anakin/Desktop/ds000113/output/1st_level/level1design/10/level1design_run_2/ev_Realign4_0_38.txt
231012-23:16:25,255 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/level1design/_subject_list_10/_level1design1/ev_Realign5_0_39.txt -> /home/anakin/Desktop/ds000113/output/1st_level/level1design/10/level1design_run_2/ev_Realign5_0_39.txt
231012-23:16:25,256 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/level1design/_subject_list_10/_level1design1/ev_Realign6_0_40.txt -> /home/anakin/Desktop/ds0001

231012-23:16:25,314 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/cope6.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/cope6.nii.gz
231012-23:16:25,323 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/cope7.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/cope7.nii.gz
231012-23:16:25,324 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/cope8.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/cope8.nii.gz
231012-23:16:25,326 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/cope9.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/model

231012-23:16:25,379 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/varcope8.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/varcope8.nii.gz
231012-23:16:25,380 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/varcope9.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/varcope9.nii.gz
231012-23:16:25,382 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/varcope10.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/varcope10.nii.gz
231012-23:16:25,383 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/varcope11.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_leve

231012-23:16:25,443 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/tstat10.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/tstat10.nii.gz
231012-23:16:25,444 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/tstat11.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/tstat11.nii.gz
231012-23:16:25,446 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/tstat12.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/tstat12.nii.gz
231012-23:16:25,448 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/tstat13.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelest

231012-23:16:25,508 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/zstat12.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/zstat12.nii.gz
231012-23:16:25,515 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/zstat13.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/zstat13.nii.gz
231012-23:16:25,518 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/zstat14.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/10/modelestimate_run_2/zstat14.nii.gz
231012-23:16:25,520 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/1st_level/modelestimate/_subject_list_10/_modelestimate1/zstat15.nii.gz -> /home/anakin/Desktop/ds000113/output/1st_level/modelest

The process ran in local memory and has traces in your temporary folder.   
Usually, the tmp folder is  cleaned by default upon reboot but if you wish 
to make room in memory without rebooting,  
copy this to command line:

sudo rm -r /tmp/*

Also, FSL has this problem of naming the its output files.  
Problem is - it outputs files in a series with the following naming convention:  xxx1,xxx2,xxx3...xxx10,xxx11..xxxN -  
where xxx can be zstat / tstat / cope / varcope.  
Since the pipeline orders it's work by the file names - we will need to rename the first 9 in that series
to a 2-digit format (i.e. 01,02,03...) in order that files will not be mixed up during 2nd level.  
We will rename all output files for each subject for each run

In [13]:
def rename_files_with_two_digit_numbers(path):
    # verify path
    if not os.path.isdir(path):
        print(f"The specified path '{path}' is not a valid directory.")
        return
    # iterate through files in the directory
    files = os.listdir(path)
    files.sort()
    for filename in files:
        file_path = os.path.join(path, filename)
        # check if it's a file and matches the specified naming convention
        if os.path.isfile(file_path) and filename.endswith('.nii.gz'):
            base_name = filename[:-len('.nii.gz')]
            # split the base name into the prefix and number
            split = list(filter(None, re.split(r'(\d+)', base_name)))
            # check if number is smaller than 10, this will ensure we're dealing with 0-9
            if int(split[1]) < 10:
                # define parts of split
                prefix = split[0]; number = split[1]
                if number.isdigit():
                    # ensure the number part has at least 2 digits
                    new_number = number.zfill(2)
                    # create the new filename
                    new_base_name = f"{prefix}{new_number}"
                    new_filename = f"{new_base_name}.nii.gz"
                    # rename the file
                    new_file_path = os.path.join(path, new_filename)
                    os.rename(file_path, new_file_path)
                    print(f"Renamed '{filename}' to '{new_filename}'")
                else:
                    print(f"Skipping '{filename}' as the number part is not a digit.")
            else:
                print(f"Skipping '{filename}' as it does not match the naming convention.")
                continue

In [14]:
# loop over all subjects
for sub in subject_list:
    # loop over all runs
    for run in range(1,num_of_runs+1):
        # check directory exists
        dir_path = os.path.join(output,'modelestimate',sub,'modelestimate_run_'+str(run)+'/')
        if not os.path.isdir(dir_path):
            print(f"The specified path '{path}' is not a valid directory.")
        else:
            # print directory and use function for renaming
            print(dir_path)
            rename_files_with_two_digit_numbers(dir_path)

/home/anakin/Desktop/ds000113/output/1st_level/modelestimate/01/modelestimate_run_1/
Renamed 'cope1.nii.gz' to 'cope01.nii.gz'
Skipping 'cope10.nii.gz' as it does not match the naming convention.
Skipping 'cope11.nii.gz' as it does not match the naming convention.
Skipping 'cope12.nii.gz' as it does not match the naming convention.
Skipping 'cope13.nii.gz' as it does not match the naming convention.
Skipping 'cope14.nii.gz' as it does not match the naming convention.
Skipping 'cope15.nii.gz' as it does not match the naming convention.
Renamed 'cope2.nii.gz' to 'cope02.nii.gz'
Renamed 'cope3.nii.gz' to 'cope03.nii.gz'
Renamed 'cope4.nii.gz' to 'cope04.nii.gz'
Renamed 'cope5.nii.gz' to 'cope05.nii.gz'
Renamed 'cope6.nii.gz' to 'cope06.nii.gz'
Renamed 'cope7.nii.gz' to 'cope07.nii.gz'
Renamed 'cope8.nii.gz' to 'cope08.nii.gz'
Renamed 'cope9.nii.gz' to 'cope09.nii.gz'
Renamed 'tstat1.nii.gz' to 'tstat01.nii.gz'
Skipping 'tstat10.nii.gz' as it does not match the naming convention.
Skipping 

In [15]:
print('1st level is done!')

1st level is done!
